# **Tabular Reinforcement Learning**

# Q-Learning on FrozenLake environment

## Non-Evaluables Practical Exercices

This is a non-evaluable practical exercise, but it is recommended that students complete it fully and individually, since it is an important part of the learning process.

The solution will be available, although it is not recommended that students consult the solution until they have completed the exercise. 

## The FrozenLake environment

In this activity, we are going to solve the [Frozen Lake](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) environment.

Main characteristics:
- The game starts with the player at location [0,0] of the frozen lake grid world with the goal located at far extent of the world e.g. [3,3] for the 4x4 environment.
- Holes in the ice are distributed in set locations when using a pre-determined map or in random locations when a random map is generated.
- The player makes moves until they reach the goal or fall in a hole.
- The lake is slippery (unless disabled) so the player may move perpendicular to the intended direction sometimes (see _is_slippery_ param).

<img src="https://gymnasium.farama.org/_images/frozen_lake.gif" />

## Q-Learning algorithm

<u>Question 1</u>: : **Implement the *Q-Learning* algorithm** using the following parameters:

- Number of episodes = 15,000
- *learning rate* = 0.8
- *discount factor* = 1

Additionally, implement de **$\epsilon$-Greedy with decay factor** method with the following parameters:
- *epsilon* = 1.0
- *max_epsilon* = 1.0
- *min_epsilon* = 0.01
- *decay_rate* = 0.005

<u>Question 2</u>: Once you have coded the algorithm, try different **values for the hyperparameters** and comment the best ones (providing an empirical comparison):

- Number of episodes
- *learning rate* 
- *discount factor* 
- *epsilon* values (including min value and decay factor)

<u>Question 3</u>: Try to solve the same environment but using a _8 x 8_ grid (also in slippery mode):
> gym.make(ENV_NAME, desc=None, map_name="8x8", is_slippery=True)

In [1]:
import gymnasium as gym

# definig the environment
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False)

print("Action space is {} ".format(env.action_space))
print("Observation space is {} ".format(env.observation_space))
print("Reward range is {} ".format(env.reward_range))

Action space is Discrete(4) 
Observation space is Discrete(16) 
Reward range is (0, 1) 


In [40]:
from collections import defaultdict
import sys
import numpy as np

def epsilon_greedy_policy(Q, state, nA, epsilon):
    '''
    Create a policy where epsilon dictates the probability of a random action being carried out.

    :param Q: linka state -> action value (dictionary)
    :param state: state in which the agent is (int)
    :param nA: number of actions (int)
    :param epsilon: possibility of random movement (float)
    :return: probability of each action (list) d
    '''
    probs = np.ones(nA) * epsilon / nA
    best_action = np.argmax(Q[state])
    probs[best_action] += 1.0 - epsilon

    return probs


def QLearning(episodes, learning_rate, discount, epsilon, decay_rate=0.005, max_epsilon = 1, min_epsilon=0.01):
    '''
    Learn to solve the environment using the Q-Learning algorithm

    :param episodes: Number of episodes (int)
    :param learning_rate: Learning rate (float [0, 1])
    :param discount: Discount factor (float [0, 1])
    :param epsilon: chance that random movement is required (float [0, 1])
    :return: x,y number of episodes and number of steps
    :Q: action value function
    '''

    # Link actions to states
    Q = np.zeros((16, 4))

    # Number of episodes
    x = np.arange(episodes) 
    y = np.zeros(episodes)

    count = 0

    for episode in range(episodes):
        
        epsilon = min (1, epsilon)

        state, _ = env.reset()
        total_reward = 0
        
        done = False
        step = 1
                
        while not done:
            probs = epsilon_greedy_policy(Q, state, env.action_space.n, epsilon)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            
            next_state, reward, terminated, truncated, _ = env.step(action) # as it's inherited from gym and not gymnasium, we receive only the done, instead of terminated and truncated. 

            # Update TD
            td_target = reward + discount * np.max(Q[next_state, :])
            td_error = td_target - Q[state][action]
            Q[state][action] += learning_rate * td_error

            done = terminated or truncated
                
            if done:
                y[episode] = step
                break
            
            total_reward += reward

            state = next_state
            #action = next_action
            step += 1
        
        epsilon = min_epsilon + (max_epsilon-min_epsilon) * np.exp(-decay_rate*count)

        if count % 100 == 0:
            print(f"Episode: {count+1} and Epsilon: {epsilon}, reward: {total_reward}")

        count += 1

    return Q

In [41]:
q = QLearning(episodes=15000, learning_rate=0.8, discount=0.95, epsilon=1)

Episode: 1 and Epsilon: 1.0, reward: 0.0
Episode: 101 and Epsilon: 0.6104653531155071, reward: 0.0
Episode: 201 and Epsilon: 0.374200646759728, reward: 0.0
Episode: 301 and Epsilon: 0.23089885854694553, reward: 0.0
Episode: 401 and Epsilon: 0.14398193040424656, reward: 0.0
Episode: 501 and Epsilon: 0.0912641486376598, reward: 0.0
Episode: 601 and Epsilon: 0.0592891976841853, reward: 0.0
Episode: 701 and Epsilon: 0.039895409588095315, reward: 0.0
Episode: 801 and Epsilon: 0.028132482499846838, reward: 0.0
Episode: 901 and Epsilon: 0.020997906572859885, reward: 0.0
Episode: 1001 and Epsilon: 0.016670567529094613, reward: 0.0
Episode: 1101 and Epsilon: 0.014045903724079427, reward: 0.0
Episode: 1201 and Epsilon: 0.012453964654899695, reward: 0.0
Episode: 1301 and Epsilon: 0.011488404801047796, reward: 0.0
Episode: 1401 and Epsilon: 0.010902763145898971, reward: 0.0
Episode: 1501 and Epsilon: 0.010547553526446355, reward: 0.0
Episode: 1601 and Epsilon: 0.010332108001623487, reward: 0.0
Epi

KeyboardInterrupt: 

In [17]:
def print_policy_SARSA(Q, width, height):
    switch_action = {
        0: "L",
        1: "D",
        2: "R",
        3: "U"
    }
    for j in range(height):
        print("------------------------------------------------------------------------")
        for i in range(width):
            arr = np.array(Q[j*width+i])
            act = np.argmax(arr)
            a = switch_action[act]
            print("  %s  |" % a, end="")
        print("")
        
    print("------------------------------------------------------------------------")

print_policy_SARSA(q, 4, 4)

------------------------------------------------------------------------
  L  |  L  |  L  |  L  |
------------------------------------------------------------------------
  L  |  L  |  L  |  L  |
------------------------------------------------------------------------
  L  |  L  |  L  |  L  |
------------------------------------------------------------------------
  L  |  L  |  L  |  L  |
------------------------------------------------------------------------


<div class="alert alert-block alert-danger">
<strong>Solution</strong>
</div>